In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\rbvan\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=\
        19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# NASA Mars News
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

In [5]:
# JPL Mars Space Images - Featured Image

url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [6]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
core_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
img_line = str(soup.select('img')[1])
img = img_line.split("=")[2][1:-3]
featured_image_url = core_url+img

In [8]:
# Mars Facts
url = 'https://space-facts.com/mars/'

Mars_df = pd.read_html('https://space-facts.com/mars/')[0]

In [9]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

''' --------------------------'''

base_url = 'https://astrogeology.usgs.gov'

hem_names = []

mydivs = soup.find_all("a", {"class": "itemLink product-item"})

x=0
for i in mydivs:
    if x % 2 == 0:
        x += 1
        continue
    else:
        i = i.text
        i = i.split()[:-1]
        i = " ".join(i)
        hem_names.append(i)
        x += 1

''' --------------------------'''


base_url = 'https://astrogeology.usgs.gov'

url_adds = []

mydivs = soup.find_all("a", {"class": "itemLink product-item"})
x = 0
for i in mydivs:
    if x%2 != 0:
        x+=1
        continue
    
    else:
        i = str(i).split(" ")[3][6:-6]
        url_adds.append(i)
        x+=1
        
urls = [str(base_url+x) for x in url_adds]


''' --------------------------'''


hemisphere_image_urls = []

x = 0
for url in urls:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    img_line = str(soup.select(".wide-image")[0])
    img = img_line.split("=")[2][1:-3]
    img = url+img
    
    hemisphere_image_urls.append(dict({'title':hem_names[x], 'img_url':img}))

    x += 1

In [11]:
scrape_dict = {'news_title':news_title,'news_p':news_p,'featured_image_url':featured_image_url,'Mars_df':Mars_df,
              'hemisphere_image_urls':hemisphere_image_urls}

In [13]:
scrape_dict

{'news_title': "NASA's Curiosity Rover Captures Shining Clouds on Mars",
 'news_p': 'The science team is studying the clouds, which arrived earlier and formed higher than expected, to learn more about the Red Planet. ',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg',
 'Mars_df':                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
 'hemisphere_image_urls': [{'title': 'Cerberus Hemisphere',
   'img_url': 'https://astrogeology.usgs.